# first try

In [21]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import xgboost
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split



X = pd.read_csv('data/X.csv')
y = pd.read_csv('data/y.csv')

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# used for kaggle submission
X_kaggle = pd.read_csv('data/X_kaggle.csv')


# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Initialize the XGBClassifier
xgb_model = xgboost.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)
best_xgb_model = grid_search.best_estimator_
print(grid_search.best_score_)

# Predict on the test set
y_pred_test = best_xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy on test set: {accuracy}")


y_pred_xgb = best_xgb_model.predict(X_kaggle)

y_pred_xgb = pd.DataFrame(y_pred_xgb, columns=['prediction'])
y_pred_xgb['ID'] = y_pred_xgb.index
y_pred_xgb



# forest = RandomForestClassifier(n_estimators=100, random_state=42)
# forest.fit(X_train, y_train)
# y_pred = forest.predict(X_test)


# y_pred = pd.DataFrame(y_pred, columns=['prediction'])
# y_pred['ID'] = y_pred.index
# y_pred

Fitting 5 folds for each of 18 candidates, totalling 90 fits
1.0
Accuracy on test set: 1.0


,prediction,ID
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
1461,0,1461
1462,0,1462
1463,0,1463
1464,0,1464


,prediction,ID
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
1461,0,1461
1462,0,1462
1463,0,1463
1464,0,1464


In [19]:
X_kaggle

,0,1,2,3,4,5,6
0,0.000000,0.000000,0.000000,30,0,0,3
1,0.185535,0.048780,0.809981,222,2,1,2
2,0.000000,0.000000,0.000000,49,0,0,5
3,0.000000,0.000000,0.000000,105,0,0,5
4,0.000000,0.000000,0.000000,35,0,1,4
...,...,...,...,...,...,...,...
1461,0.000000,0.000000,0.000000,49,0,0,3
1462,0.066667,0.062500,0.369269,70,1,1,2
1463,0.000000,0.000000,0.000000,24,0,0,5
1464,0.000000,0.000000,0.000000,153,0,0,5


In [20]:
# this file should be submitted
y_pred_xgb.to_csv('data/y_pred.csv', index=False)